# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
S. Kumar  ->  S. Kumar  |  ['S. Kumar']
T. Henning  ->  T. Henning  |  ['T. Henning']
Y. Reinarz  ->  Y. Reinarz  |  ['Y. Reinarz']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


Arxiv has 64 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.18955


extracting tarball to tmp_2510.18955...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.18965


extracting tarball to tmp_2510.18965...

...

 done.


T. Henning  ->  T. Henning  |  ['T. Henning']
L. Acuña  ->  L. Acuña  |  ['L. Acuña']
Y. Reinarz  ->  Y. Reinarz  |  ['Y. Reinarz']


Found 82 bibliographic references in tmp_2510.19162/sample701.bbl.
Retrieving document from  https://arxiv.org/e-print/2510.19584


extracting tarball to tmp_2510.19584...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.19162-b31b1b.svg)](https://arxiv.org/abs/2510.19162) | **A Compact Multi-Planet System of Three Transiting Giant Planets Around TIC118798035**  |
|| R. Brahm, et al. -- incl., <mark>T. Henning</mark>, <mark>Y. Reinarz</mark> |
|*Appeared on*| *2025-10-23*|
|*Comments*| *17 pages, submitted to AAS journals*|
|**Abstract**|            We report the discovery and characterization of three transiting giant planets in the TIC118798035 system. The three planets were identified as transiting candidates from data of the TESS mission, and confirmed with ground-based photometric transit observations along with radial velocity variations obtained with FEROS, HARPS and ESPRESSO. The three planets present transit timing variations (TTVs). We performed a N-body orbital fitting to the TTVs and radial velocities finding that TIC118798035 b is as warm low-density Neptune with a mass of 0.0250$\pm$0.0023 $M_J$, a radius of 0.655$\pm$0.018 $R_J$, and an orbital period of 11.507 d; TIC118798035 c is a warm Saturn with a mass of 0.403$\pm$0.024 $M_J$, a radius of 0.973$\pm$0.023 $R_J$, and an orbital period of 22.564 d; and TIC118798035 d is a warm Jupiter with a mass of 0.773$\pm$0.052 $M_J$, a radius of 0.923$\pm$0.044 $R_J$, and an orbital period of 48.925 d. The bulk metallicities of the three planets don't fully follow the mass-metallicity correlation found for the giant planets of the solar system, which hints at a somewhat different formation history for the planets of the TIC118798035 system. TIC118798035 is the only system having more than two transiting planets larger than 0.5 $R_J$ with a precise orbital and physical characterization, amenable for future atmospheric studies.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.18955-b31b1b.svg)](https://arxiv.org/abs/2510.18955) | **Intruder Alert: Breaking Resonant Chains with Planetesimal Flybys**  |
|| <mark>J. Li</mark>, C. E. O'Connor, F. A. Rasio |
|*Appeared on*| *2025-10-23*|
|*Comments*| *Lead jointly by JL and CEO, 16 pages, 7 figures, 2 tables, submitted to ApJL*|
|**Abstract**|            The orbital architectures of compact exoplanet systems record their complicated dynamical histories. Recent research supports the ``breaking-the-chains'' hypothesis, which proposes that compact systems typically form in chains of mean-motion resonances (MMRs) but subsequently break out on a $\sim 100$Myr timescale. We investigate a scenario for breaking the chains through intermittent flybys of planetesimals originating from a distant reservoir. Using $N$-body simulations and semi-analytical calculations, we characterize the disruption of MMRs through these flybys. We find a planetesimal reservoir of total mass $\gtrsim 0.04 M_{\oplus}$ is required to disrupt MMR chains, depending on the mass distribution and the typical number of flybys executed by each planetesimal. We verify that systems disrupted in this way are frequently unstable to close encounters within $\sim 100$Myr of the final flyby. This mechanism operates in systems with both a sufficiently massive reservoir and an efficient mechanism for planetesimal injection. Consequently, we predict an anti-correlation between resonant inner systems and dynamically active outer configurations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.18965-b31b1b.svg)](https://arxiv.org/abs/2510.18965) | **Intermediate-Mass Stripped Stars in the Magellanic Clouds: Forward Modeling the Observed Population Discovered Via UV Excess**  |
|| L. Blomberg, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-10-23*|
|*Comments*| *29 pages, 19 figures, submitted to PASP*|
|**Abstract**|            Stripped stars are hot, helium-rich stars formed when binary interactions remove a star's hydrogen envelope. While low-mass ($\lesssim 1\,M_\odot$) and high-mass ($\gtrsim 8\,M_\odot$) stripped stars are well studied as hot subdwarfs and Wolf-Rayet stars, their intermediate-mass counterparts ($1-8 \,M_\odot$) have only recently been discovered. The Stripped-Star Ultraviolet Magellanic Cloud Survey (SUMS) identified UV-excess sources in the Magellanic Clouds using Swift-UVOT photometry and selected 820 photometric stripped-star candidates. However, the selection function, completeness, and purity of this sample remain poorly understood. We forward model the population of stripped stars in the Magellanic Clouds using a binary population synthesis model combined with spatially resolved star formation histories and simulated UV photometry. To assess survey sensitivity, we inject simulated sources into real Swift-UVOT images and reproduce the SUMS selection process, including crowding, extinction, and photometric quality cuts. We recover $\sim 250$ simulated stripped stars with masses $\gtrsim 1\,M_{\odot}$, which corresponds to a recover rate of $9-13\,\%$. The rest are missed due to dilution by luminous companions, crowding, and high extinction. The observed population is biased toward systems with low-mass companions formed by common envelope evolution and toward systems with compact object companions. Of the stripped stars which show UV excess, $25-45\,\%$ are identifiable by SUMS; higher-resolution data or improved reddening corrections are needed to detect the rest. We predict contamination of the observed stripped star candidates by main-sequence stars with spurious UV excess due to crowding and provide guidelines for selecting higher-purity subsamples. These results enable tests of binary evolution models and realistic comparison of observed and simulated stripped-star populations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.19010-b31b1b.svg)](https://arxiv.org/abs/2510.19010) | **Constraints on Axion-Like Particles from VERITAS Observations of a Flaring Radio Galaxy in the Perseus Cluster**  |
|| C. B. Adams, et al. -- incl., <mark>S. Kumar</mark> |
|*Appeared on*| *2025-10-23*|
|*Comments*| *18 pages, 6 figures, accepted for publication in the Physical Review D (PRD)*|
|**Abstract**|            Background: Axion-like particles (ALPs) are hypothetical particles that emerge in numerous theoretical extensions to the Standard Model. Their coupling to electromagnetic field implies that ALPs would mix with photons in the presence of external magnetic fields. As ALP phenomenology is governed by the mass and strength of its coupling, there is a subset of this parameter space in which this mixing would be expected to leave an imprint on the spectra of TeV gamma-ray sources. Data: In 2017, the VERITAS gamma-ray observatory recorded the second day of a dramatic flare of the radio galaxy NGC 1275, embedded at the center of the Perseus galaxy cluster. This serendipitous locale provides a spatially-extended magnetic field of strength O(10$\mu$G) through which escaping photons traverse, making it an excellent target to study ALPs. Methods: We analyze the VERITAS data of NGC 1275's 2017 flare with the gammapy analysis package. Extensive fitting and modeling are performed to ultimately conduct a likelihood analysis used to search for any evidence of a preference for ALPs and to explore the confidence with which constraints can be set. We adopt the CLs method for this study for its conservative approach to setting limits in regimes where the search has limited sensitivity. Results: No evidence for the existence of ALPs is found, and no combination of mass and coupling strength can be excluded at or above 95% confidence level. We provide a map showing the strength of our exclusions in the mass and coupling parameter space. The strongest exclusions are found in the mass range $2 \times 10^{-7}$eV $\lesssim m_a \lesssim 4 \times 10^{-7}$eV and at the coupling strength of $g_{a\gamma} \gtrsim 3 \times 10^{-11}$ GeV$^{-1}$ up to 80% confidence level, which are consistent with previous studies. Conclusions: We find the CLs method to be a trustworthy approach, and advocate for its...         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.19584-b31b1b.svg)](https://arxiv.org/abs/2510.19584) | **Accretion with two-phase gas supply and its application in black hole X-ray binaries**  |
|| <mark>Y. Wang</mark>, B. F. Liu, M. Liu |
|*Appeared on*| *2025-10-23*|
|*Comments*| *10 pages, 6 figures; accepted for publication in MNRAS*|
|**Abstract**|            Accretion in black hole X-ray binaries is commonly believed to be supplied by the Roche lobe overflow or the stellar wind. The former is thought to form a geometrically thin disc while the diffuse wind could form a geometrically thick hot accretion flow. In this paper, we instead consider a more generalised case, i.e., accretion with both cold and hot gas supplies, which feed a disc and a corona respectively. We investigate the interaction of disc and corona by analysing the energy coupling and matter exchange, i.e. corona condensation/disc evaporation, with a semi-analytical method. It is found that the accretion geometry in the radial direction and the resultant emission spectrum depend strongly on both the total gas supply rate and the ratio of cold and hot gases. For gas supply rates of a few percent of the Eddington value, diverse geometries and spectral shapes are possible, depending on the fraction of cold gas supply. This provides an interpretation for the various spectra observed in intermediate states. However, at higher accretion rates, regardless of the form of the feeding gas, the inner accretion flow is always disc-dominated, implying an inevitable transition to the soft state, while at very low gas supply rates, hard state spectrum dominated by the hot flow is expected. We also present the predicted hardness-intensity correlation of Cygnus X-1, and constrain the value of the viscosity parameter of the accretion flow to the range of 0.25--0.35 by comparing our results with MAXI observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.19162/./TIC118798035_planetb.png', 'tmp_2510.19162/./a_evol.png', 'tmp_2510.19162/./e_evol.png', 'tmp_2510.19162/./i_evol.png', 'tmp_2510.19162/./P1_evol.png', 'tmp_2510.19162/./P2_evol.png', 'tmp_2510.19162/./Delta_w_evol.png', 'tmp_2510.19162/./periodogram.png', 'tmp_2510.19162/./periodogram_all.png']
copying  tmp_2510.19162/./TIC118798035_planetb.png to _build/html/
copying  tmp_2510.19162/./a_evol.png to _build/html/
copying  tmp_2510.19162/./e_evol.png to _build/html/
copying  tmp_2510.19162/./i_evol.png to _build/html/
copying  tmp_2510.19162/./P1_evol.png to _build/html/
copying  tmp_2510.19162/./P2_evol.png to _build/html/
copying  tmp_2510.19162/./Delta_w_evol.png to _build/html/
copying  tmp_2510.19162/./periodogram.png to _build/html/
copying  tmp_2510.19162/./periodogram_all.png to _build/html/
exported in  _build/html/2510.19162.md
    + _build/html/tmp_2510.19162/./TIC118798035_planetb.png
    + _build/html/tmp_2510.19162/./a_evol.png
    + _build

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\rPb}{11.507 d}$
$\newcommand{\rPc}{22.564 d}$
$\newcommand{\rPd}{48.925 d}$
$\newcommand{\rpl}{R_P}$
$\newcommand{\mpl}{M_P}$
$\newcommand{\rjup}{R_J}$
$\newcommand{\mjup}{M_J}$
$\newcommand{\rsun}{R_{\odot}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\rpb}{0.655\pm0.018 \rjup}$
$\newcommand{\rpc}{0.973\pm0.023 \rjup}$
$\newcommand{\rpd}{0.923\pm0.044 \rjup}$
$\newcommand{\mpb}{0.0250\pm0.0023 \mjup}$
$\newcommand{\mpc}{0.403\pm0.024 \mjup}$
$\newcommand{\mpd}{0.773\pm0.052 \mjup}$
$\newcommand{\thetable}{A\arabic{table}}$
$\newcommand{\thefigure}{A\arabic{figure}}$</div>



<div id="title">

# A Compact Multi-Planet System of Three Transiting Giant Planets Around TIC118798035

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.19162-b31b1b.svg)](https://arxiv.org/abs/2510.19162)<mark>Appeared on: 2025-10-23</mark> -  _17 pages, submitted to AAS journals_

</div>
<div id="authors">

R. Brahm, et al. -- incl., <mark>T. Henning</mark>, <mark>L. Acuña</mark>, <mark>Y. Reinarz</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery and characterization of three transiting giant planets in the $\stname$ system. The three planets were identified as transiting candidates from data of the TESS mission, and confirmed with ground-based photometric transit observations along with radial velocity variations obtained with FEROS, HARPS and ESPRESSO. The three planets present transit timing variations (TTVs). We performed a N-body orbital fitting to the TTVs and radial velocities finding that $\plnameb$ is as warm low-density Neptune with a mass of $\mpb$ , a radius of $\rpb$ , and an orbital period of $\rPb$ ; $\plnamec$ is a warm Saturn with a mass of $\mpc$ , a radius of $\rpc$ , and an orbital period of $\rPc$ ; and $\plnamed$ is a warm Jupiter with a mass of $\mpd$ , a radius of $\rpd$ , and an orbital period of $\rPd$ . The bulk metallicities of the three planets don't fully follow the mass-metallicity correlation found for the giant planets of the solar system, which hints at a somewhat different formation history for the planets of the $\stname$ system. $\stname$ is the only system having more than two transiting planets larger than 0.5 $R_J$ with a precise orbital and physical characterization, amenable for future atmospheric studies.

</div>

<div id="div_fig1">

<img src="tmp_2510.19162/./TIC118798035_planetb.png" alt="Fig2" width="100%"/>

**Figure 2. -** TESS and ground-based transits of \plnameb centered on the transit time predicted from linear ephemeris. The transit model generated by  \texttt{juliet} is also plotted for each transit.  (*fig:transitsb*)

</div>
<div id="div_fig2">

<img src="tmp_2510.19162/./a_evol.png" alt="Fig10.1" width="16%"/><img src="tmp_2510.19162/./e_evol.png" alt="Fig10.2" width="16%"/><img src="tmp_2510.19162/./i_evol.png" alt="Fig10.3" width="16%"/><img src="tmp_2510.19162/./P1_evol.png" alt="Fig10.4" width="16%"/><img src="tmp_2510.19162/./P2_evol.png" alt="Fig10.5" width="16%"/><img src="tmp_2510.19162/./Delta_w_evol.png" alt="Fig10.6" width="16%"/>

**Figure 10. -** Orbital evolution of the TIC 118798035 system for an extent of 1000 years for the best-fit (maximum $-lnL$) sample from the MCMC posteriors.
    *Top panels, from left to right:* evolution of the planetary semi-major axes, eccentricities and inclinations where \plnameb is shown with blue
    \plnamec with red, and \plnamed with green, respectively.
    *Bottom panels, from left to right* show the evolution of the period ratio between the inner pair $P_{\rm{c}} / P_{\rm{b}}$, the outer pair $P_{\rm{d}} / P_{\rm{c}}$, and the libration of the apsidal angle $\Delta\omega_{c-d}$ in aligned configuration around 0$^\circ$.
     (*evol_plot*)

</div>
<div id="div_fig3">

<img src="tmp_2510.19162/./periodogram.png" alt="Fig1.1" width="50%"/><img src="tmp_2510.19162/./periodogram_all.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** GLS periodogram generated from the FEROS, HARPS, and ESPRESSO radial velocities of \stname. The horizontal lines correspond to the 10\% and 1\% false alarm probabilities. The red and blue dashed vertical lines mark the orbital periods of candidates $b$ and $c$, respectively. The green dashed vertical lines mark the possible period aliases of \plnamed based on TESS photometry.Same as in Figure \ref{fig:gls} but for the bisector spans, FWHM, and S-index. Here the green dashed line corresponds to the orbital period of \plnamed constrained by the radial velocities and HATPI light curve. (*fig:gls*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.19162"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

114  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
